In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("Reviews.csv")
data = data.sample(frac=1)[:20000]

In [3]:
data.columns = map(lambda x:x.lower(), list(data))
data["text"] = data["summary"] + " "+ data["text"]
data = data[["text", "score"]]

data.loc[data.score<3, "score"] = -1
data.loc[data.score==3, "score"] = 0
data.loc[data.score>3, "score"] = 1

data.head(5)


,text,score
52748,Outstanding Quality I was shocked how quickly ...,1
296117,My only sweetener I am crazy about Splenda. I...,1
212956,Worst dont even buy them I hate these ....they...,-1
442106,These are the best! The only way to improve on...,1
109152,Chili Peppers Seed Kit not very fruitful Today...,0


In [4]:
data.tail(5)

,text,score
109520,Yummy This exquisite coffee arrived in perfect...,1
160162,very good Had this when i lived in TX and know...,1
561144,"Choking hazard Seemed like a great idea, and m...",-1
231625,A Legend Among The Junk Food Realm Have you ev...,1
242014,Only flavor I did not like at all I like all o...,-1


In [5]:
import random
import numpy
sentiment_data = zip(data["text"], data["score"])
s = list(sentiment_data)
print(s[1])

('My only sweetener I am crazy about Splenda.  I mix equal parts Splenda And Brown sugar for my hot cerael.  It saves money and calories.  Asparteem  ( Nutrasweet)gives me loose bowels.  Splenda meets my sweetner needs without calories!', 1)


In [6]:
random.shuffle(s)

# 80% for training
train_X, train_y = zip(*s[:16000])

# Keep 20% for testing
test_X, test_y = zip(*s[16000:])
test_X

("These taste great I read an interview in Inc magazine about the founder of the company, and it inspired me to try a bar at Starbucks, where they are selling them at the cash register these days. I loved it. The one I sampled was different from this product, but this one sounded like I would enjoy it even more, and I did.<br /><br />These bars are not particularly low calorie, though, in the article I mention above, the founder says he sometimes eats two of them for lunch.  That's about right, it would be a tasty light on-the-go lunch at 420 calories.  So much better for you than a candy bar.",
 'Careful of your expectations I hoped to be different, but like other reviewers I did not recieve a very good mix of "samples". I recieved 6 mints, 8 earl grays, etc leaving me with just a few to try instead of 10 different kinds like i hoped. Luckily, i really did like the mint so it sorta kinda worked out in the end.',
 'Use this!!! I use Star Kay White chocolate extract in everything from b

In [7]:
from nltk import sent_tokenize
data
text = data.loc[475425, "text"]
print(text)
raw_sentences = sent_tokenize(text)

raw_sentences

KeyError: 'the label [475425] is not in the [index]'

In [ ]:
from nltk import word_tokenize

sentence = raw_sentences[0]
words = word_tokenize(sentence)
words


In [ ]:
from nltk import pos_tag

tags = pos_tag(words)
tags


In [ ]:
import nltk
#nltk.download('wordnet')
#nltk.download('sentiwordnet')

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

wn_tag = penn_to_wn(tags[0][1])
word = tags[0][0]
lemmatizer = WordNetLemmatizer()
lemma = lemmatizer.lemmatize(word, pos=wn_tag)

lemma


In [ ]:
synsets = wn.synsets(lemma, pos=wn_tag)
synsets
print(synsets)
from nltk.corpus import sentiwordnet as swn
synset = synsets[0]
swn_synset = swn.senti_synset(synset.name())

print("Positive score = "+ str(swn_synset.pos_score()))
print("Negative score = "+ str(swn_synset.neg_score()))

sentiment = swn_synset.pos_score() - swn_synset.neg_score()
print("Sentiment = "+ str(sentiment))

In [ ]:
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import sent_tokenize, word_tokenize, pos_tag

lemmatizer = WordNetLemmatizer()

def sentiment_sentiwordnet(text):
    raw_sentences = sent_tokenize(text)
    sentiment = 0
    tokens_count = 0
    
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            word_sent = swn_synset.pos_score() - swn_synset.neg_score()

            if word_sent != 0:
                sentiment += word_sent
                tokens_count += 1

    if tokens_count == 0:
        return 0
    sentiment = sentiment/tokens_count
    if sentiment >= 0.01:
        return 1
    if sentiment <= -0.01:
        return -1
    return 0



In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
pred_y = [sentiment_sentiwordnet(text) for text in test_X]
accuracy_score(test_y, pred_y)